In [1]:
import cv2
import pandas as pd
import numpy as np
import time
import math
import threading
import keyboard

from sklearn.linear_model import LinearRegression

import DobotDllType as dType  # dobot control

source1 = 0
CON_STR = {
    dType.DobotConnect.DobotConnect_NoError:  "DobotConnect_NoError",
    dType.DobotConnect.DobotConnect_NotFound: "DobotConnect_NotFound",
    dType.DobotConnect.DobotConnect_Occupied: "DobotConnect_Occupied"}
api = dType.load()
 
#Connect Dobot
state = dType.ConnectDobot(api, "", 115200)[0]
print("Connect status:",CON_STR[state])

The dll you use is 64-bit, in order to run smoothly, please ensure that your python environment is also 64-bit
The python environment is: ('64bit', 'WindowsPE')
Connect status: DobotConnect_NoError


In [114]:
dType.DisconnectDobot(api)

In [42]:
# dType.ClearAllAlarmsState(api)

# dType.SetHOMEParams(api,150,0,0,0,0)
# dType.SetQueuedCmdStopExec(api)
# dType.SetPTPCmd(api, dType.PTPMode.PTPMOVLXYZINCMode, 0, 0, 20, 0, isQueued = 0)
# dType.SetQueuedCmdStartExec(api)
dType.SetEndEffectorSuctionCup(api,1,0,0)

# dType.SetPTPCommonParams(api, 75, 75, isQueued = 1)


[0]

In [2]:
#finding the blue color using open cv and some threshold
def findblue(frame):
    blue = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    bluelow = np.array([120,50,50])#replace with your HSV Values
    bluehi = np.array([130,255,255])#replace with your HSV Values
    mask = cv2.inRange(blue, bluelow, bluehi)
    res = cv2.bitwise_and(frame, frame, mask=mask)
    cnts, hir = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if len(cnts) > 0 :
        maxcontour = max(cnts, key = cv2.contourArea)
        #finding center coordinates of the color
        M = cv2.moments(maxcontour)
        if M['m00'] > 0 and cv2.contourArea(maxcontour) > 2000:
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            return (cx, cy), True
        else:
            #320,240 center of the camera image, when there is no color
            return (int(frame.shape[1]/2),int(frame.shape[0]/2)), False
    else:
        return (int(frame.shape[1]/2),int(frame.shape[0]/2)), False


In [3]:
def commandDelay(lastIndex):
    dType.SetQueuedCmdStartExec(api)
    while lastIndex > dType.GetQueuedCmdCurrentIndex(api)[0]:
        dType.dSleep(200)
    dType.SetQueuedCmdStopExec(api)

In [4]:
# function to move the dobot to the predicted x, y location based on click on the mapped screen
def MoveDobot(predDoboX,predDoboY):
    # dType.SetQueuedCmdStopExec(api)
    # dType.SetQueuedCmdClear(api)
    dType.ClearAllAlarmsState(api)
    last_index = dType.SetPTPCmd(api, 1, predDoboX, predDoboY, 90, 0, isQueued = 1)[0]
    # dType.SetQueuedCmdStartExec(api)
    commandDelay(last_index)
    # skipframes = True # for solving lag in frames

In [5]:
def MoveDobotInstantly(predDoboX,predDoboY):
    dType.SetQueuedCmdStopExec(api)
    dType.ClearAllAlarmsState(api)
    dType.SetQueuedCmdClear(api)
    print(f"Sending dobot to position{predDoboX},{predDoboY}")
    dType.SetPTPCmd(api, 1, predDoboX, predDoboY, 90, 0, isQueued = 0)
    dType.SetQueuedCmdStartExec(api)
    dType.ClearAllAlarmsState(api)



In [21]:
def moveDoboSmoothly(x,y):
    print(f"moving dobot to point ({x},{y})")
    dType.ClearAllAlarmsState(api)
    last_index = dType.SetPTPCmd(api, 1, x, y, 0, 0, isQueued = 0)[0]
    commandDelay(last_index)

In [6]:
def Grab():
   
    # dType.SetQueuedCmdStopExec(api)
    print("GRAB started")
    dType.ClearAllAlarmsState(api)
    


    # last_index1 = dType.SetPTPCmd(api, dType.PTPMode.PTPMOVLXYZINCMode, 0, 0, -25, 0, isQueued = 0)[0]
    # print(f"last_index for going down = {last_index1}")
    # commandDelay(last_index1)
    # # dType.SetQueuedCmdStartExec(api)    

    dType.SetWAITCmd(api,1000,1)
    
    # suck
    dType.SetEndEffectorSuctionCup(api,1,1,0)
    # commandDelay(last_index)
    dType.SetWAITCmd(api,1000,1)

    # come up dobot going up
    dType.SetQueuedCmdStopExec(api)
    dType.ClearAllAlarmsState(api)
    dType.SetQueuedCmdClear(api)
    print(f"Moving dobot up z direction downwards for grabbing")
    dType.SetPTPCmd(api, 7, 0, 0, 30, 0, isQueued = 0)
    dType.SetQueuedCmdStartExec(api)
    dType.ClearAllAlarmsState(api)

    dType.SetWAITCmd(api,1000,1)


In [68]:
unGrab()
# dType.ClearAllAlarmsState(api)

Starting Ungrab
Moving dobot up in z direction for ungrabbing
Completing ungrab


In [77]:
# def MoveUp():
   
#     # dType.SetQueuedCmdStopExec(api)
#     print("GRAB started")
#     dType.ClearAllAlarmsState(api)
    

    

IndentationError: expected an indented block (Temp/ipykernel_28300/2129480295.py, line 3)

In [7]:
def unGrab():

    print("Starting Ungrab")
    dType.ClearAllAlarmsState(api)




    # last_index1 = dType.SetPTPCmd(api,1,0, 150, 30, 0, 0)[0]
    # print("sending dobot to drop location")
    # print(f"last_index1 above operation= {last_index1}")
    # commandDelay(last_index1)

    dType.SetWAITCmd(api,2000,1)

    # suck off
    dType.SetEndEffectorSuctionCup(api,1,0,0)[0]
    print("Completing ungrab")
    # print(f"last_index for suck off ={last_index}")
    # commandDelay(last_index)
    dType.SetWAITCmd(api,1000,1)



In [8]:
# function to display the coordinates of the points clicked on the image 

def click_event(event, x, y, flags, params):
  
    # checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
      # Predict the dobot XY points based on the screen XY clicked points
     
      Pred_DobotXYZ = reg.predict([[x,y]])[0]
      print('predicted Dobot X,Y = ', reg.predict([[x,y]])[0])
      
      # move dobot to the point
      MoveDobotInstantly(Pred_DobotXYZ[0],Pred_DobotXYZ[1])
      
      
      # displaying the coordinates on the image window
      font = cv2.FONT_HERSHEY_SIMPLEX 
      cv2.putText(img, str(x) + ',' + str(y), (x,y), font,1, (255, 0, 0), 2)
      cv2.imshow('Image', img)

    # checking for Right mouse clicks
    if event == cv2.EVENT_RBUTTONDOWN:
        
        # displaying the coordinates on the Shell
        print("Right Click")
        print('ScreenX = ', x, ' ScreenY = ', y)
        
        # add click point x,y to the list 
        ScreenXY.append((x,y))
        
        DobotPose = dType.GetPose(api)
        print(f"DoboX= {DobotPose[0]}, DoboY = {DobotPose[1]}",)
        DobotXYZ.append(DobotPose[:2])

        # displaying the coordinates on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, str(x) + ',' + str(y), (x,y), font,1, (255, 0, 0), 2)
        cv2.imshow('Image', img)



In [9]:
DobotXYZ = [] # list to store the dobot xy for corresponding screen click point
ScreenXY = [] # list of Screen Click (x pixel , y pixel) vector 

In [15]:
dType.SetQueuedCmdClear(api)
dType.SetQueuedCmdStopExec(api)
dType,dType.SetHOMEParams(api,200,10,90,0,0)
dType.SetQueuedCmdStartExec(api)
dType.SetHOMECmd(api,0,0)

[2365, 0]

In [14]:
dType.ClearAllAlarmsState(api)

In [16]:
 # Main program
 
cap = cv2.VideoCapture(1)   
cap.set(3, 1920)
cap.set(4, 1080)

# variables 
pTime = 0
cTime = 0
prev_DoboXY = [150,0]
global skipframes
# skipframes = True

pausefollow = False


# print(dType.GetAlarmsState(api))

while True:
    success, img = cap.read()
    frame = img.copy()
    # print(f"size of img = {img.shape[1]},{img.shape[0]}")

    # Calculating frames per second
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    
    # display fps on top left
    cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3,(255, 0, 255), 3)

    # click event for mapping screen xy to dobot xy using linear regression
    # left click for ptp motion and right click to record screen and dobot xy for mapping 
    cv2.namedWindow('Image',cv2.WINDOW_FULLSCREEN)
    cv2.setMouseCallback('Image', click_event)

    # get blue object center coordinates and put on the img 
    (bluex, bluey), blogic = findblue(img)
    cv2.putText(img, str(bluex) + " , " + str (bluey), (bluex - 100 ,bluey - 30), cv2.FONT_HERSHEY_PLAIN, 3,(0, 255, 0), 3)
 
    # draw red circle on the blue object center
    red = [0,0,255]
    cv2.circle(img,(bluex,bluey),20,red,2)
   
    # print("skipframes = ",skipframes)
    dType.ClearAllAlarmsState(api)

    # if blue object is present
    if blogic: 

        # predict dobot x,y based on blue object screen x,y using reg
        Pred_DobotXYZ = reg.predict([[bluex,bluey]])[0]
        
        # check if the current blue objectg center is at a significant distance from its previous position
        deltaBlue = math.dist(Pred_DobotXYZ, prev_DoboXY)
        # print(f"deltaX = {deltaX} and deltaY = {deltaY}")
        cv2.putText(img, "deltaX = " + str(round(deltaBlue,2)), (10,120), cv2.FONT_HERSHEY_PLAIN, 2,(0, 255, 0), 2)
        
        if deltaBlue > 10 and pausefollow == False:
            # skipframes = False # for solving lag in frames

            # send dobot to object
            # dType.SetQueuedCmdClear(api)
            # dType.ClearAllAlarmsState(api)
            p1 = threading.Thread(target=MoveDobotInstantly,args=[Pred_DobotXYZ[0],Pred_DobotXYZ[1]])
            p1.start()
            p1.join()
            # moveDoboSmoothly(Pred_DobotXYZ[0],Pred_DobotXYZ[1])
            # dType.SetWAITCmd(api,1000,1)

            # Grab()
            # dType.SetWAITCmd(api,1000,1)
            
            # unGrab()
            # dType.SetWAITCmd(api,1000,1)
            
            prev_DoboXY = Pred_DobotXYZ
        
    if keyboard.is_pressed('g'):
        dType.ClearAllAlarmsState(api)
        print(" you pressed g for grabbing")
        p2 = threading.Thread(target=Grab)
        p2.start()

    if keyboard.is_pressed('d'):
        dType.ClearAllAlarmsState(api)
        print("moving dobot to destination")
        p4 = threading.Thread(target=MoveDobotInstantly,args = [-100,200])
        p4.start()

    if keyboard.is_pressed('w'):
        #  dobot going down
        pausefollow = True
        dType.SetQueuedCmdStopExec(api)
        dType.ClearAllAlarmsState(api)
        dType.SetQueuedCmdClear(api)
        print(f"Moving dobot in z direction up by 30 mm")
        dType.SetPTPCmd(api, 7, 0, 0, 30, 0, isQueued = 0)
        dType.SetQueuedCmdStartExec(api)
        dType.ClearAllAlarmsState(api)
        pausefollow = True


    if keyboard.is_pressed('s'):
        #  dobot going down
        dType.SetQueuedCmdStopExec(api)
        dType.ClearAllAlarmsState(api)
        dType.SetQueuedCmdClear(api)
        print(f"Moving dobot in z direction downwards by 30 mm")
        dType.SetPTPCmd(api, 7, 0, 0, -30, 0, isQueued = 0)
        dType.SetQueuedCmdStartExec(api)
        dType.ClearAllAlarmsState(api)
        pausefollow = True

    if keyboard.is_pressed('u'):
        dType.ClearAllAlarmsState(api)
        print(" you pressed g for grabbing")
        p3 = threading.Thread(target=unGrab)
        p3.start()
    
    if keyboard.is_pressed('r'):
        pausefollow == False


        # else:
        #     p1.join()
    #show image 
    cv2.imshow("Image", img)

    if cv2.waitKey(1) & 0xFF == 13:  # close window when enter is pressed
        dType.ClearAllAlarmsState(api)
        break

cv2.destroyAllWindows()
cap.release()

Sending dobot to position61.53852726828433,47.96886348617494
Sending dobot to position50.75829805999456,47.66323451953099
Sending dobot to position62.365672875124574,44.34647410118134
Sending dobot to position53.6623029685133,50.58121870115673
Moving dobot in z direction up by 30 mm
Moving dobot in z direction up by 30 mm
Sending dobot to position63.608143613549544,41.95113944494477
Sending dobot to position57.39415441513802,51.092113266290426
Sending dobot to position67.3369576913559,37.19573501323839
Sending dobot to position54.906876500735564,51.83178335042487
Sending dobot to position65.26617312731435,41.18795944029921
Sending dobot to position54.07318886874788,44.11137489607063
Moving dobot in z direction downwards by 30 mm
Moving dobot in z direction downwards by 30 mm
Moving dobot in z direction up by 30 mm
Sending dobot to position69.41054598046065,38.06470966018372
Sending dobot to position56.97555827097972,44.193659617859396
Moving dobot in z direction downwards by 30 mm
Send

In [40]:
dType.DisconnectDobot(api)

In [13]:
DobotXYZ

[[246.27597045898438, -160.95997619628906],
 [139.01065063476562, -212.9073028564453],
 [206.52716064453125, 7.7322516441345215],
 [318.00830078125, 23.6977596282959],
 [291.40545654296875, 165.14584350585938],
 [113.01554870605469, 188.7032928466797]]

In [14]:
ScreenregXY

[(1271, 355), (1377, 584), (922, 431), (912, 200), (619, 220), (535, 594)]

In [15]:
reg = LinearRegression().fit(ScreenXY, DobotXYZ)
print(reg.score(ScreenXY, DobotXYZ))
print(reg.coef_)
print(reg.intercept_)

0.9988735590988629
[[ 0.02688019 -0.48397412]
 [-0.48301414 -0.03736792]]
[386.09010163 470.46078415]


In [16]:
# save regressionn mapping in csv file
Dobotxyz_copy = DobotXYZ.copy()
Screenxyz_copy = ScreenXY.copy()

reg_dobot_points = pd.DataFrame(Dobotxyz_copy,columns=['DoboX','DoboY'])
reg_screen_points = pd.DataFrame(Screenxyz_copy,columns=['ScreenX','ScreenY'])
reg_points_df = reg_screen_points.join(reg_dobot_points)
print(reg_points_df)
# print(reg_dobot_points)
# print(reg_screen_points)

reg_points_df.to_csv('RegPointSaved.csv')


   ScreenX  ScreenY       DoboX       DoboY
0     1271      355  246.275970 -160.959976
1     1377      584  139.010651 -212.907303
2      922      431  206.527161    7.732252
3      912      200  318.008301   23.697760
4      619      220  291.405457  165.145844
5      535      594  113.015549  188.703293


In [10]:
# Use the previous saved regression and calibration to run dobot PTP

saved_reg_points = pd.read_csv('RegPointSaved.csv', index_col = 0)
saved_reg_points[['ScreenX','ScreenY']]
reg = LinearRegression().fit(saved_reg_points[['ScreenX','ScreenY']], saved_reg_points[['DoboX','DoboY']])
reg.score(saved_reg_points[['ScreenX','ScreenY']],saved_reg_points[['DoboX','DoboY']])

0.9993241844904308

In [22]:
dType.DisconnectDobot(api)


In [11]:
def commandDelay(lastIndex):
    dType.SetQueuedCmdStartExec(api)
    while lastIndex > dType.GetQueuedCmdCurrentIndex(api)[0]:
        dType.dSleep(200)
    dType.SetQueuedCmdStopExec(api)

In [72]:
i = dType.GetQueuedCmdCurrentIndex(api)[0]
type(i)


int

In [75]:
ex = dType.SetPTPCmd(api, 1, 150, 0, -20, 0, isQueued = 1)[0]
type(ex)


int

In [27]:

if __name__ == '__main__':
    p1 = multiprocessing.Process(target=Grab)
    p1.start()



In [31]:
# dType.SetEndEffectorSuctionCup(api,1,0,0)
unGrab()

Ungrab
last_index1 for destination= 51
last_index for suck off =0


In [22]:
i = 0
p = []
while i < 3:
    p1 = multiprocessing.Process(target=MoveDobot,args=[180,0])
    p.append(p1)        
    # # moveDoboSmoothly(180,0)
    # p2 = multiprocessing.Process(target=Grab)
    
    # p3 = multiprocessing.Process(target=MoveDobot,args=[240,50])
    # # moveDoboSmoothly(200,40)

    # p4 = multiprocessing.Process(target=unGrab)
    # unGrab()
    # dType.SetWAITCmd(api,1000)
    dType.ClearAllAlarmsState(api)
    p1.start()
    # p2.start()
    # p3.start()
    i = i + 1
for process in p:
    process.join()

In [29]:
dType.SetEndEffectorSuctionCup(api,1,0,0)

[0]

In [32]:
import threading
if __name__ == '__main__':
    p1 = threading.Thread(target=MoveDobotInstantly,args=(180,40))
    p1.start()
    p1.join()
    print("done...")
    


Sending dobot to position180,40
done...
